In [33]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import numpy as np
import os
import time

In [34]:
team = 'boston-celtics'
wait = 5

In [36]:
driver = webdriver.Chrome()
time.sleep(wait)
driver.get(f'https://www.spotrac.com/nba/{team}/yearly/cap/')
driver.implicitly_wait(10)
print("successful entrance")
def year_breakdown(yr):
    """
    yr a list of 2-3 elements containing a player's salary cap data for that year
    returns: salary in dollar amount, percentage of total cap
    """
    if "UFA" in yr or "RFA" in yr or "Two-Way" in yr:
        return np.nan, np.nan
    if "Ext. Eligible" in yr:
        yr.remove("Ext. Eligible")
    if len(yr) == 3:
        return int(yr[0].replace('$', '').replace(',','')), float(yr[2].replace('%',''))
    elif len(yr) == 2:
        return int(yr[0].replace('$', '').replace(',','')), float(yr[1].replace('%',''))
    else:
        return np.nan, np.nan
    
data = []
try: 
    table = driver.find_element(By.XPATH, '//*[@id="main"]/div/div[3]/div[2]/table[1]/tbody')
    rows = table.find_elements(By.XPATH, '//tr[@role="row"]') # get all table rows
    flag = 0
    for row in rows:
        if "PLAYERS" in row.text:  # break if trying to access other tables than Active Roster
            if flag == 0:
                flag = 1
            else:
                break
        try:  # access player name, contract type in first column cells
            player = row.find_elements(By.CLASS_NAME, 'player')
            name, contract, type = [elem.strip() for elem in player[0].text.split('\n')]
        except Exception as e:
            print("Failed at extracting player name and contract type data")
            print(e)
            continue
        try:
            info = row.find_elements(By.CLASS_NAME, 'center')
            pos, age, yr1, yr2, yr3, yr4, yr5, yr6 = [elem.text.split('\n') for elem in info[:8]] # create individual lists
            pos = str(pos[0])
            age = int(age[0])
            sal1, per1 = year_breakdown(yr1)
            sal2, per2 = year_breakdown(yr2)
            sal3, per3 = year_breakdown(yr3)
            sal4, per4 = year_breakdown(yr4)
            sal5, per5 = year_breakdown(yr5)
            sal6, per6 = year_breakdown(yr6)
        except Exception as e:
            print("Failed at extracting yearly contract data")
            print(e)
            continue
        print(f"{name}, {contract}, {type}, {pos}, {age}, {sal1, per1}, {sal2, per2}, {sal3, per3}, {sal4, per4}, {sal5, per5}, {sal6, per6}")
        data.append([name, contract, type, pos, age, sal1, per1, sal2, per2, sal3, per3, sal4, per4, sal5, per5, sal6, per6])
        print("----------------------------------------------------")
    driver.quit()
except Exception as e:
    driver.quit()
    print("Failed to get webpage")
    print(e)


successful entrance
Failed at extracting player name and contract type data
not enough values to unpack (expected 3, got 1)
Jrue Holiday, 4 yr, $134,997,333 (AAV: $33.75M), Extension / Veteran, PG, 33, (36861707, 27.1), (39403893, 27.9), (nan, nan), (nan, nan), (nan, nan), (nan, nan)
----------------------------------------------------
Kristaps Porzingis, 5 yr, $158,253,000 (AAV: $31.65M), Free Agent / Maximum, PF, 28, (36016200, 26.5), (29268293, 20.8), (30731707, 19.8), (nan, nan), (nan, nan), (nan, nan)
----------------------------------------------------
Jayson Tatum, 5 yr, $163,000,300 (AAV: $32.60M), Extension / Designated Rookie, PF, 25, (32600060, 24.0), (34848340, 24.7), (37096620, 23.9), (nan, nan), (nan, nan), (nan, nan)
----------------------------------------------------
Jaylen Brown, 4 yr, $106,333,334 (AAV: $26.58M), Extension / Rookie, SF, 27, (31830357, 23.4), (49350000, 35.0), (53298000, 34.4), (57246000, 33.6), (61194000, 32.6), (65142000, 31.6)
---------------------

In [30]:
df = pd.DataFrame(data, columns=['Name', 'Contract', 'Type', 'Position', 'Age', 'Year 1 Salary', 'Year 1 %Cap', 'Year 2 Salary', 'Year 2 %Cap', 'Year 3 Salary', 'Year 3 %Cap', 'Year 4 Salary', 'Year 4 %Cap', 'Year 5 Salary', 'Year 5 %Cap', 'Year 6 Salary', 'Year 6 %Cap'])

In [31]:
df['Team'] = team

In [32]:
df

,Name,Contract,Type,Position,Age,Year 1 Salary,Year 1 %Cap,Year 2 Salary,Year 2 %Cap,Year 3 Salary,Year 3 %Cap,Year 4 Salary,Year 4 %Cap,Year 5 Salary,Year 5 %Cap,Year 6 Salary,Year 6 %Cap,Team
0,Trae Young,"5 yr, $215,159,700 (AAV: $43.03M)",Extension / Designated Rookie,PG,25,40064220.0,29.5,43031940.0,30.5,45999660.0,29.7,48967380.0,28.7,NaN,NaN,NaN,NaN,atlanta-hawks
1,De'Andre Hunter,"4 yr, $90,000,000 (AAV: $22.50M)",Extension / Rookie,SF,26,20089286.0,14.8,21696429.0,15.4,23303571.0,15.0,24910714.0,14.6,NaN,NaN,NaN,NaN,atlanta-hawks
2,Bogdan Bogdanovic,"4 yr, $68,000,000 (AAV: $17.00M)",Extension / Veteran,SG,31,18700000.0,13.7,17260000.0,12.2,16020000.0,10.3,16020000.0,9.4,NaN,NaN,NaN,NaN,atlanta-hawks
3,Dejounte Murray,"4 yr, $64,000,000 (AAV: $16.00M)",Extension / Rookie,PG,27,18214000.0,13.4,25499599.0,18.1,27539568.0,17.8,29523536.0,17.3,31507504.0,16.8,NaN,NaN,atlanta-hawks
4,Onyeka Okongwu,"4 yr, $26,422,143 (AAV: $6.61M)",1st-Round / Rookie Scale,C,23,8109063.0,6.0,14000000.0,9.9,15000000.0,9.7,16100000.0,9.4,16880000.0,9.0,NaN,NaN,atlanta-hawks
5,Saddiq Bey,"4 yr, $13,030,303 (AAV: $3.26M)",1st-Round / Rookie Scale,SF,24,4556983.0,3.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,atlanta-hawks
6,Kobe Bufkin,"4 yr, $19,801,203 (AAV: $4.95M)",1st-Round / Rookie Scale,PG,20,4094280.0,3.0,4299000.0,3.0,4503720.0,2.9,NaN,NaN,NaN,NaN,NaN,NaN,atlanta-hawks
7,A.J. Griffin,"4 yr, $17,106,137 (AAV: $4.28M)",1st-Round / Rookie Scale,SF,20,3712920.0,2.7,3889920.0,2.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,atlanta-hawks
8,Jalen Johnson,"4 yr, $12,888,585 (AAV: $3.22M)",1st-Round / Rookie Scale,SF,22,2925360.0,2.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,atlanta-hawks
9,Bruno Fernando,"4 yr, $10,869,564 (AAV: $2.72M)",Extension / Veteran,C,25,2581522.0,1.9,2717391.0,1.9,2853260.0,1.8,NaN,NaN,NaN,NaN,NaN,NaN,atlanta-hawks


In [14]:
d = os.getcwd()
df.to_csv(os.path.join(d, f'{team}-contracts.csv'), index=False)

In [13]:
print(d)

C:\Users\jrh25\PycharmProjects\NBA-contracts


In [15]:
test = pd.read_csv(f'{team}-contracts.csv')
test

,Name,Contract,Type,Position,Age,Year 1 Salary,Year 1 %Cap,Year 2 Salary,Year 2 %Cap,Year 3 Salary,Year 3 %Cap,Year 4 Salary,Year 4 %Cap,Year 5 Salary,Year 5 %Cap,Year 6 Salary,Year 6 %Cap,Team
0,Trae Young,"5 yr, $215,159,700 (AAV: $43.03M)",Extension / Designated Rookie,PG,25,40064220.0,29.5,43031940.0,30.5,45999660.0,29.7,48967380.0,28.7,NaN,NaN,NaN,NaN,atlanta-hawks
1,De'Andre Hunter,"4 yr, $90,000,000 (AAV: $22.50M)",Extension / Rookie,SF,26,20089286.0,14.8,21696429.0,15.4,23303571.0,15.0,24910714.0,14.6,NaN,NaN,NaN,NaN,atlanta-hawks
2,Bogdan Bogdanovic,"4 yr, $68,000,000 (AAV: $17.00M)",Extension / Veteran,SG,31,18700000.0,13.7,17260000.0,12.2,16020000.0,10.3,16020000.0,9.4,NaN,NaN,NaN,NaN,atlanta-hawks
3,Dejounte Murray,"4 yr, $64,000,000 (AAV: $16.00M)",Extension / Rookie,PG,27,18214000.0,13.4,25499599.0,18.1,27539568.0,17.8,29523536.0,17.3,31507504.0,16.8,NaN,NaN,atlanta-hawks
4,Onyeka Okongwu,"4 yr, $26,422,143 (AAV: $6.61M)",1st-Round / Rookie Scale,C,23,8109063.0,6.0,14000000.0,9.9,15000000.0,9.7,16100000.0,9.4,16880000.0,9.0,NaN,NaN,atlanta-hawks
5,Saddiq Bey,"4 yr, $13,030,303 (AAV: $3.26M)",1st-Round / Rookie Scale,SF,24,4556983.0,3.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,atlanta-hawks
6,Kobe Bufkin,"4 yr, $19,801,203 (AAV: $4.95M)",1st-Round / Rookie Scale,PG,20,4094280.0,3.0,4299000.0,3.0,4503720.0,2.9,NaN,NaN,NaN,NaN,NaN,NaN,atlanta-hawks
7,A.J. Griffin,"4 yr, $17,106,137 (AAV: $4.28M)",1st-Round / Rookie Scale,SF,20,3712920.0,2.7,3889920.0,2.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,atlanta-hawks
8,Jalen Johnson,"4 yr, $12,888,585 (AAV: $3.22M)",1st-Round / Rookie Scale,SF,22,2925360.0,2.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,atlanta-hawks
9,Bruno Fernando,"4 yr, $10,869,564 (AAV: $2.72M)",Extension / Veteran,C,25,2581522.0,1.9,2717391.0,1.9,2853260.0,1.8,NaN,NaN,NaN,NaN,NaN,NaN,atlanta-hawks
